# Same Atom Selection
----
- Same chemical context
- Iterate the same context within one frame
----
- Expect minor changes

In [13]:
import open3d as o3d
import time, builtins, tempfile, datetime, os 
from BetaPose import utils, chemtools, representations; 

import pytraj as pt 
import numpy as np 
from scipy.stats import entropy 
from scipy.ndimage import gaussian_filter 
from scipy.spatial import distance_matrix 

from BetaPose import utils, cluster

FEATURIZER_PARMS = {
  # Mask of components 
  "MASK_INTEREST" : ":LIG,MDL", 
  "MASK_ENVIRONMENT" : ":1-221",
  
  # POCKET SETTINGS
  "VOXEL_DIMENSION" : [12, 12, 12],    # Unit: 1 (Number of lattice in one dimension)
  "CUBOID_LENGTH" : [8,8,8],           # Unit: Angstorm (Need scaling)
  
  # SEARCH SETTINGS
  "UPDATE_INTERVAL" : 1, 
  "CUTOFF": 18, 
}

# Example workflow

from BetaPose import trajloader, data_io
from BetaPose import features, featurizer


# Load multiple trajectories
# trajs = "/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_001_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_002_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_003_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_004_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_005_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_006_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_007_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_008_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_009_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_010_traj.nc%"
# tops = ["/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_010_END.pdb"] * 10


trajs = "/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_001_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_002_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_003_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_004_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_005_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_006_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_007_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_008_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_009_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_010_traj.nc%"
tops = ["/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_008_END.pdb"] * 10
trajs = trajs.strip("%").split("%")
tloader = trajloader.TrajectoryLoader(trajs, tops);
trajectories = trajloader.TrajectoryLoader(trajs, tops); 

for traj in trajectories: 
  # Complete the trajectory information
  traj.strip(":T3P")
  print(traj.traj)
  #### traj.addcharge(); 
  
  # Initialize the featurizer since different trajectory might have distinct parameters
  # featurizer  = featurizer.Featurizer3D(FEATURIZER_PARMS); 
  featurizer  = featurizer.Featurizer3D(FEATURIZER_PARMS); 
  feature_mass = features.MassFeature(); 
  # NOTE: in this step, the feature hooks back the feature and could access the featurizer by feat.featurer
  
  featurizer.register_feature(feature_mass)   # i features
  featurizer.register_traj(traj)
  featurizer.register_frames(range(5,100))
  # featurizer.register_frames(range(1,50,10))  # j frames
  # featurizer.register_centers() # k centers 
  
  repr_traji, fpfh_traji, features_traji = featurizer.run_by_atom(traj.top.select("@CA&:5"), fbox_length=[6,6,6])
  
#   print(repr_traji.tolist())
  
  print(repr_traji.shape)
  print(fpfh_traji.shape)
  print(features_traji.shape)
#   featurizer.dump("repr_form", repr_traji, "/tmp/test.h5"); 
#   featurizer.dump("FPFH", np.array([(0,d) for d in fpfh_traji], dtype=object), "/tmp/test.h5"); 
  break
  
import pickle 
thedict = {
  "repr":repr_traji, 
  "fpfh":fpfh_traji, 
  "features":features_traji
}

with open('/tmp/test.pkl', 'wb') as f:
  # Write the object to the file
  pickle.dump(thedict, f)

pytraj.Trajectory, 1001 frames: 
Size: 0.056316 (GB)
<Topology: 2517 atoms, 188 residues, 38 mols, PBC with box type = cubic>
           
23-05-22T14:49:02   : [6 6 6]
23-05-22T14:49:02   : Frame 5: Generated 1 centers
23-05-22T14:49:02   : Found 4 segments (array([0, 1, 2, 3]), array([2472,    5,   35,    5]))
[2, 1, 3]
23-05-22T14:49:02   : Segment 1 / 3: TriangleMesh with 1039 points and 2074 triangles.
23-05-22T14:49:02   : Viewpoint: [24.18283061 17.08332724 40.19961598]; Sum of VPC is: 1.0
23-05-22T14:49:02   : Segment 2 / 3: TriangleMesh with 231 points and 458 triangles.
23-05-22T14:49:02   : Viewpoint: [27.32909957 14.1884026  39.81405628]; Sum of VPC is: 1.0000000000000002
23-05-22T14:49:02   : Final round VPC 3/3; Lookback viewpoint: [20.69376154 18.45812692 37.83131154] -> 1.0000000000000002
23-05-22T14:49:02   : Number of objects for lookback viewpoint component: 2
23-05-22T14:49:02   : Segment 3 / 3: TriangleMesh with 260 points and 516 triangles.
[[3.5e+01 8.0e+00 6.0e+0

23-05-22T14:49:04   : Segment 1 / 2: TriangleMesh with 986 points and 1968 triangles.
23-05-22T14:49:04   : Viewpoint: [24.84741785 15.54924746 39.98127282]; Sum of VPC is: 0.9999999999999999
23-05-22T14:49:04   : Final round VPC 2/2; Lookback viewpoint: [20.74118846 17.24943462 37.28854615] -> 0.9999999999999999
23-05-22T14:49:04   : Number of objects for lookback viewpoint component: 1
23-05-22T14:49:04   : Segment 2 / 2: TriangleMesh with 260 points and 516 triangles.
[[3.1e+01 9.0e+00 4.0e+00 ... 1.0e-02 1.0e-02 0.0e+00]
 [5.0e+00 1.0e+00 0.0e+00 ... 2.0e-02 1.0e-02 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]]
23-05-22T14:49:04   : Final 3D object:  TriangleMesh with 1246 points and 2484 triangles.
23-05-22T14:49:04   : Centers 1 ; Feature vector:  (1, 1, 12, 12, 12)
(1, 252) (1, 33, 600) (1, 1, 12, 

23-05-22T14:49:06   : Viewpoint: [23.77285438 16.69852357 39.4586431 ]; Sum of VPC is: 0.9999999999999999
23-05-22T14:49:06   : Final round VPC 2/2; Lookback viewpoint: [19.21848881 18.45731343 37.10245522] -> 1.0
23-05-22T14:49:06   : Number of objects for lookback viewpoint component: 1
23-05-22T14:49:06   : Segment 2 / 2: TriangleMesh with 268 points and 532 triangles.
[[3.9e+01 1.1e+01 7.0e+00 ... 1.0e-02 0.0e+00 0.0e+00]
 [5.0e+00 2.0e+00 0.0e+00 ... 2.0e-02 1.0e-02 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]]
23-05-22T14:49:06   : Final 3D object:  TriangleMesh with 1456 points and 2904 triangles.
23-05-22T14:49:06   : Centers 1 ; Feature vector:  (1, 1, 12, 12, 12)
(1, 252) (1, 33, 600) (1, 1, 12, 12, 12)
23-05-22T14:49:06   : Frame 20: Generated 1 centers
23-05-22T14:49:06   : Found 3 segments (a

23-05-22T14:49:09   : Viewpoint: [24.00970418 16.20976992 40.46845817]; Sum of VPC is: 1.0000000000000002
23-05-22T14:49:09   : Final round VPC 2/2; Lookback viewpoint: [20.11215232 17.68636424 37.59671854] -> 0.9999999999999997
23-05-22T14:49:09   : Number of objects for lookback viewpoint component: 1
23-05-22T14:49:09   : Segment 2 / 2: TriangleMesh with 302 points and 600 triangles.
[[3.3e+01 9.0e+00 5.0e+00 ... 2.0e-02 0.0e+00 1.0e-02]
 [7.0e+00 2.0e+00 0.0e+00 ... 2.0e-02 1.0e-02 1.0e-02]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]]
23-05-22T14:49:09   : Final 3D object:  TriangleMesh with 1306 points and 2604 triangles.
23-05-22T14:49:09   : Centers 1 ; Feature vector:  (1, 1, 12, 12, 12)
(1, 252) (1, 33, 600) (1, 1, 12, 12, 12)
23-05-22T14:49:09   : Frame 28: Generated 1 centers
23-05-22T14:49:09   : Foun

23-05-22T14:49:11   : Viewpoint: [22.71997148 17.40349951 40.40322812]; Sum of VPC is: 0.9999999999999999
23-05-22T14:49:11   : Final round VPC 2/2; Lookback viewpoint: [19.20692459 18.89202295 37.67915738] -> 0.9999999999999999
23-05-22T14:49:11   : Number of objects for lookback viewpoint component: 1
23-05-22T14:49:11   : Segment 2 / 2: TriangleMesh with 305 points and 606 triangles.
[[3.2e+01 1.0e+01 5.0e+00 ... 1.0e-02 0.0e+00 0.0e+00]
 [8.0e+00 2.0e+00 0.0e+00 ... 3.0e-02 1.0e-02 1.0e-02]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]]
23-05-22T14:49:11   : Final 3D object:  TriangleMesh with 1322 points and 2636 triangles.
23-05-22T14:49:11   : Centers 1 ; Feature vector:  (1, 1, 12, 12, 12)
(1, 252) (1, 33, 600) (1, 1, 12, 12, 12)
23-05-22T14:49:11   : Frame 36: Generated 1 centers
23-05-22T14:49:11   : Foun

...

23-05-22T14:49:20   : Segment 1 / 2: TriangleMesh with 1211 points and 2418 triangles.
23-05-22T14:49:20   : Viewpoint: [23.40873328 17.53993724 40.08339141]; Sum of VPC is: 1.0
23-05-22T14:49:20   : Final round VPC 2/2; Lookback viewpoint: [19.18671545 18.63254201 37.59618428] -> 1.0
23-05-22T14:49:20   : Number of objects for lookback viewpoint component: 1
23-05-22T14:49:20   : Segment 2 / 2: TriangleMesh with 369 points and 734 triangles.
[[3.7e+01 1.2e+01 4.0e+00 ... 2.0e-02 1.0e-02 1.0e-02]
 [9.0e+00 3.0e+00 0.0e+00 ... 1.0e-02 1.0e-02 1.0e-02]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]]
23-05-22T14:49:20   : Final 3D object:  TriangleMesh with 1580 points and 3152 triangles.
23-05-22T14:49:20   : Centers 1 ; Feature vector:  (1, 1, 12, 12, 12)
(1, 252) (1, 33, 600) (1, 1, 12, 12, 12)
23-05-22T14:49:20   :

23-05-22T14:49:22   : Segment 1 / 2: TriangleMesh with 1194 points and 2384 triangles.
23-05-22T14:49:22   : Viewpoint: [24.64947069 17.1414598  40.14827387]; Sum of VPC is: 0.9999999999999999
23-05-22T14:49:22   : Final round VPC 2/2; Lookback viewpoint: [20.41068439 18.25121262 37.50793355] -> 1.0
23-05-22T14:49:22   : Number of objects for lookback viewpoint component: 1
23-05-22T14:49:22   : Segment 2 / 2: TriangleMesh with 301 points and 598 triangles.
[[4.2e+01 1.0e+01 7.0e+00 ... 1.0e-02 0.0e+00 0.0e+00]
 [5.0e+00 2.0e+00 0.0e+00 ... 1.0e-02 1.0e-02 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]]
23-05-22T14:49:22   : Final 3D object:  TriangleMesh with 1495 points and 2982 triangles.
23-05-22T14:49:22   : Centers 1 ; Feature vector:  (1, 1, 12, 12, 12)
(1, 252) (1, 33, 600) (1, 1, 12, 12, 12)
23-05-

23-05-22T14:49:25   : Segment 1 / 2: TriangleMesh with 1081 points and 2158 triangles.
23-05-22T14:49:25   : Viewpoint: [25.00586031 16.40183626 40.25418501]; Sum of VPC is: 0.9999999999999999
23-05-22T14:49:25   : Final round VPC 2/2; Lookback viewpoint: [20.97828723 17.9316844  38.13555319] -> 1.0
23-05-22T14:49:25   : Number of objects for lookback viewpoint component: 1
23-05-22T14:49:25   : Segment 2 / 2: TriangleMesh with 282 points and 560 triangles.
[[3.5e+01 9.0e+00 5.0e+00 ... 1.0e-02 0.0e+00 0.0e+00]
 [6.0e+00 2.0e+00 0.0e+00 ... 1.0e-02 1.0e-02 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]]
23-05-22T14:49:25   : Final 3D object:  TriangleMesh with 1363 points and 2718 triangles.
23-05-22T14:49:25   : Centers 1 ; Feature vector:  (1, 1, 12, 12, 12)
(1, 252) (1, 33, 600) (1, 1, 12, 12, 12)
23-05-

23-05-22T14:49:27   : Segment 1 / 2: TriangleMesh with 1236 points and 2468 triangles.
23-05-22T14:49:27   : Viewpoint: [24.42569903 16.64285437 39.71574838]; Sum of VPC is: 1.0
23-05-22T14:49:27   : Final round VPC 2/2; Lookback viewpoint: [20.02045175 18.18900439 37.52832018] -> 1.0
23-05-22T14:49:27   : Number of objects for lookback viewpoint component: 1
23-05-22T14:49:27   : Segment 2 / 2: TriangleMesh with 228 points and 452 triangles.
[[3.9e+01 9.0e+00 6.0e+00 ... 0.0e+00 1.0e-02 0.0e+00]
 [3.0e+00 0.0e+00 0.0e+00 ... 1.0e-02 1.0e-02 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]]
23-05-22T14:49:27   : Final 3D object:  TriangleMesh with 1464 points and 2920 triangles.
23-05-22T14:49:27   : Centers 1 ; Feature vector:  (1, 1, 12, 12, 12)
(1, 252) (1, 33, 600) (1, 1, 12, 12, 12)
23-05-22T14:49:27   :

23-05-22T14:49:29   : Segment 1 / 2: TriangleMesh with 1241 points and 2478 triangles.
23-05-22T14:49:29   : Viewpoint: [23.80736664 16.28055923 39.88864061]; Sum of VPC is: 0.9999999999999998
23-05-22T14:49:29   : Final round VPC 2/2; Lookback viewpoint: [19.92052717 17.47494022 38.00245109] -> 1.0000000000000002
23-05-22T14:49:29   : Number of objects for lookback viewpoint component: 1
23-05-22T14:49:29   : Segment 2 / 2: TriangleMesh with 184 points and 364 triangles.
[[3.8e+01 1.0e+01 6.0e+00 ... 2.0e-02 0.0e+00 1.0e-02]
 [2.0e+00 0.0e+00 0.0e+00 ... 1.0e-02 1.0e-02 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]]
23-05-22T14:49:29   : Final 3D object:  TriangleMesh with 1425 points and 2842 triangles.
23-05-22T14:49:29   : Centers 1 ; Feature vector:  (1, 1, 12, 12, 12)
(1, 252) (1, 33, 600) (1, 1, 12,

In [14]:
from scipy.spatial import distance
import matplotlib.pyplot as plt 
import numpy as np 


def compute_similarity(array1, array2):
    array1 = np.array(array1).reshape(-1)
    array2 = np.array(array2).reshape(-1)
    
    min_1 = np.min(array1)
    min_2 = np.min(array2)
    max_1 = np.max(array1)
    max_2 = np.max(array2)
    normalized_array1 = (array1 - min_1) / (max_1 - min_1)
    normalized_array2 = (array2 - min_2) / (max_2 - min_2)
#     normalized_array1 = (array1)
#     normalized_array2 = (array2)
    
#     dot_product = np.dot(normalized_array1, normalized_array2)
#     norm_1 = np.linalg.norm(normalized_array1)
#     norm_2 = np.linalg.norm(normalized_array2)
#     similarity = dot_product / (norm_1 * norm_2)
#     similarity = (similarity + 1) / 2

    distance = np.sqrt(np.sum((normalized_array1 - normalized_array2) ** 2))
    # Convert distance to similarity (assuming max possible distance is sqrt(12))
#     print(distance)
    similarity = max(1 - (distance / np.sqrt(1/12)), 0)
    
    return similarity

# repr_simi = np.zeros((len(fpfh_traji), len(fpfh_traji)))
# for i in range(len(repr_traji)): 
#   for j in range(len(repr_traji)): 
#     # diff = repr_traji[i] - repr_traji[j]
#     sim = compute_similarity(repr_traji[i], repr_traji[j])
#     repr_simi[i][j] = sim
# #     print(sim)


# fig, ax = plt.subplots()
# ax.hist(repr_simi.reshape(-1))
# fig.show()
# fig, ax = plt.subplots()
# ax.imshow(repr_simi, cmap="inferno")
# fig.show()

for i in repr_traji: 
  print(compute_similarity(repr_traji[0], i))
  
  
  
  

1.0
0
0
0
0
0
0
0
0
0
0
0
0.11604924461904309
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.1940784319826453
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


# Same Center 
----
- Similar chemical context since trajectory is aligned
- Iterate different frames with one center 
----
- Expect minor changes

# Different Atom Selection
----
- Different chemical context
- Iterate different context within one frame
----
- Expect significant change

In [3]:
x = [np.nan, np.nan, np.nan]


# Viewpoint Feature histogram


In [ ]:
import numpy as np

def compute_vfh(points, normals, viewpoint, bins=45):
  # Compute the centroid
  centroid = np.mean(points, axis=0); 
  # Compute the direction from the centroid to each point
  directions = viewpoint - centroid; 
  # Compute the angles between the directions and the normals
  angles = np.arccos(np.sum(normals * directions, axis=1) / np.linalg.norm(directions, axis=1))
  # Compute the viewpoint component histogram
  vfh_hist_viewpoint, _ = np.histogram(angles, bins=bins, range=(0, np.pi))

  # Compute the Extended FPFH component for each point
  extended_fpfh = np.zeros((len(points), bins))

  for i, (point, normal) in enumerate(zip(points, normals)):
      k_neighbors = np.arange(len(points))
      k_neighbors = k_neighbors[k_neighbors != i]
      extended_fpfh[i] = compute_pfh(points, normals, i, k_neighbors, bins)

  # Compute the Extended FPFH histogram by summing up the histograms for all points
  vfh_hist_extended = np.sum(extended_fpfh, axis=0)

  # Normalize the histograms
  vfh_hist_viewpoint = vfh_hist_viewpoint / np.sum(vfh_hist_viewpoint)
  vfh_hist_extended = vfh_hist_extended / np.sum(vfh_hist_extended)

  # Concatenate the viewpoint and extended histograms to form the VFH
  vfh = np.concatenate((vfh_hist_viewpoint, vfh_hist_extended))

  return vfh


In [5]:
import numpy as np 
import open3d as o3d
import time
from BetaPose import utils

from matplotlib.cm import inferno

def calc_tm(roll, pitch, yaw, translate=[0, 0, 0]):
    # Precompute trigonometric functions
    cos_roll, sin_roll = np.cos(roll), np.sin(roll)
    cos_pitch, sin_pitch = np.cos(pitch), np.sin(pitch)
    cos_yaw, sin_yaw = np.cos(yaw), np.sin(yaw)

    # Generate rotation matrices
    Rx = np.array([[1, 0, 0], [0, cos_roll, -sin_roll], [0, sin_roll, cos_roll]])
    Ry = np.array([[cos_pitch, 0, sin_pitch], [0, 1, 0], [-sin_pitch, 0, cos_pitch]])
    Rz = np.array([[cos_yaw, -sin_yaw, 0], [sin_yaw, cos_yaw, 0], [0, 0, 1]])

    # Combine rotations
    R = Rx @ Ry @ Rz

    # Create the final transformation matrix
    H = np.eye(4); 
    H[:3, :3] = R; 
    H[:3, 3] = np.array(translate).ravel()

    return H


def transform_pcd(pcd, trans_mtx): 
  # Homogenize the point cloud (add a row of ones)
  homogeneous_pcd = np.hstack((pcd, np.ones((pcd.shape[0], 1))));
  # Apply the transformation matrix to the point cloud
  transformed_pcd = np.dot(homogeneous_pcd, trans_mtx.T);
  # Remove the homogeneous coordinate (last column)
  transformed_pcd = transformed_pcd[:, :3];
  return transformed_pcd
  

pcd = np.arange(102*3).reshape(-1,3)
# pcd = np.asarray([np.zeros(102),np.zeros(102), np.arange(102)]).T
print(pcd)
pcd_out = np.array(pcd)

t1 = time.perf_counter()

for r in np.linspace(0, np.pi*2, 20): 
  for p in np.linspace(0, np.pi*2, 20): 
    for z in np.linspace(0, np.pi*2, 20): 
      TransMatrix = utils.calc_tm(r, p , z)
      pcd2 = utils.transform_pcd(pcd, TransMatrix)
      pcd_out = np.vstack((pcd_out, pcd2))
  
  
print("Method1: ", time.perf_counter() - t1,)
# print(pcd2)


# import matplotlib.pyplot as plt 

point_cloud = o3d.geometry.PointCloud()

# Set the points of the point cloud object
point_cloud.points = o3d.utility.Vector3dVector(pcd_out)

thecmap = inferno(np.linspace(0,1,len(pcd_out)))[:, :3]
print(thecmap)

point_cloud.colors = o3d.utility.Vector3dVector(thecmap)


# Visualize the point cloud
o3d.visualization.draw_geometries([point_cloud])




[[  0   1   2]
 [  3   4   5]
 [  6   7   8]
 ...
 [297 298 299]
 [300 301 302]
 [303 304 305]]
Method1:  6.0915879950043745
[[1.46200e-03 4.66000e-04 1.38660e-02]
 [1.46200e-03 4.66000e-04 1.38660e-02]
 [1.46200e-03 4.66000e-04 1.38660e-02]
 ...
 [9.88362e-01 9.98364e-01 6.44924e-01]
 [9.88362e-01 9.98364e-01 6.44924e-01]
 [9.88362e-01 9.98364e-01 6.44924e-01]]


In [10]:
pcd = np.arange(102*3).reshape(-1,3)
np.vstack((pcd,pcd))

array([[  0,   1,   2],
       [  3,   4,   5],
       [  6,   7,   8],
       [  9,  10,  11],
       [ 12,  13,  14],
       [ 15,  16,  17],
       [ 18,  19,  20],
       [ 21,  22,  23],
       [ 24,  25,  26],
       [ 27,  28,  29],
       [ 30,  31,  32],
       [ 33,  34,  35],
       [ 36,  37,  38],
       [ 39,  40,  41],
       [ 42,  43,  44],
       [ 45,  46,  47],
       [ 48,  49,  50],
       [ 51,  52,  53],
       [ 54,  55,  56],
       [ 57,  58,  59],
       [ 60,  61,  62],
       [ 63,  64,  65],
       [ 66,  67,  68],
       [ 69,  70,  71],
       [ 72,  73,  74],
       [ 75,  76,  77],
       [ 78,  79,  80],
       [ 81,  82,  83],
       [ 84,  85,  86],
       [ 87,  88,  89],
       [ 90,  91,  92],
       [ 93,  94,  95],
       [ 96,  97,  98],
       [ 99, 100, 101],
       [102, 103, 104],
       [105, 106, 107],
       [108, 109, 110],
       [111, 112, 113],
       [114, 115, 116],
       [117, 118, 119],
       [120, 121, 122],
       [123, 124

In [7]:
_pcd = np.arange(100).reshape((-1,4))
c = np.mean(_pcd, axis = 0); 
print(c)

[48. 49. 50. 51.]


In [32]:
import matplotlib.pyplot as plt 
import time


class VFH(object):
  """
  Parent class for PFH
  """

  def __init__(self, div, nneighbors, rad):
    """
    Pass in parameters
    """
    self._div = div
    self._nneighbors = nneighbors
    self._radius = rad

    self._error_list = []
    self._Rlist = []
    self._tlist = []
    self._pcd = np.array([])
    print("After initializaiton");

  def push_pcd(self, pcd, norm):
    if len(self._pcd) > 0:
      self._pcd = np.vstack(self._pcd, pcd);
      self._norm = np.vstack(self._norm, norm);
    else:
      self._pcd = np.array(pcd);
      self._norm = np.array(norm);
    self.N = len(self._pcd);
    self._kdtree = spatial.KDTree(self._pcd);

  def get_neighbors(self, pq):
    """
    Get k nearest neighbors of the query point pq from pc within the radius using KDTree.

    :param pq: Query point (1D numpy array of shape (3,))
    :param pc: Point cloud (2D numpy array of shape (N, 3))
    :return: Indices of the k nearest neighbors within the radius, distances of the k nearest neighbors
    """

    distances, indices = self._kdtree.query(pq, k=self._nneighbors+1, distance_upper_bound=self._radius)
    # Filter out neighbors with infinite distance (outside the search radius)
    valid_neighbors = np.isfinite(distances)
    indices = indices[valid_neighbors]
    distances = distances[valid_neighbors]
    return distances, indices

  def compute_fpfh(self):
    
    pass
  
  def compute_efpfh(self):
    pcd_center = np.mean(self._pcd, axis=0);

    for point_idx in range(self.N):
      pi = self._pcd[point_idx];
      ni = self._norm[point_idx];
      
      _, neighbor_idx = self.get_neighbors(pi);
      neighbors = neighbor_idx.tolist(); 
      neighbors.remove(point_idx);    # Remove the original query point
      # print(f"#### {len(neighbor_idx)} points queried")
      # print(point_idx, "=>" , neighbors)
      for nb_idx in neighbors: 
        u = self._norm[nb_idx]; 
        v = np.cross(pi-pcd_center, u)
        w = np.cross(u, v)
        dist = np.linalg.norm(vc)

        alpha = np.dot(v, u)
        phi = np.dot(u, vc)
        theta = np.arctan(np.dot(w, u) / np.dot(u, u))
        
  def compute_vpc(self, viewpoint, bins=128): 
    
    # Compute the relative position of the viewpoint to each point in the cloud
    rel_viewpoint = np.asarray(viewpoint) - self._pcd.mean(axis=0)
    print(rel_viewpoint)

    # Normalize the relative viewpoint vectors
    rel_viewpoint_normalized = rel_viewpoint / np.linalg.norm(rel_viewpoint) #[:, np.newaxis]
    print(rel_viewpoint_normalized, rel_viewpoint_normalized.shape)

    # Calculate the angle between the normals and the relative viewpoint vectors
    cos_angles = np.sum(self._norm * rel_viewpoint_normalized, axis=1)
    angles = np.arccos(cos_angles)
    
    print(np.dot(self._norm, rel_viewpoint_normalized))
    for i in self._norm: 
      print(np.dot(i, rel_viewpoint_normalized))

    # Create the viewpoint component histogram
    hist, _ = np.histogram(angles, bins=bins, range=(0, np.pi))

    # Normalize the histogram
    hist_normalized = hist / np.sum(hist)
#     plt.plot(np.arange(len(hist_normalized)), hist_normalized)
#     plt.show()
    return hist_normalized

        


div = 2
nneighbors = 8
rad = 0.8

import open3d as o3d
import numpy as np
from scipy import spatial

mesh = o3d.io.read_triangle_mesh("/media/yzhang/MieT5/BetaPose/tests/test.ply")

pcd = np.array(mesh.vertices)
# [1200:1800]
norms = np.array(mesh.vertex_normals)
# [1200:1800]
print(pcd.shape, norms.shape)

vfh_container = VFH(div, nneighbors, rad)
vfh_container.push_pcd(pcd, norms); 
print(pcd)

# vfh_container.calc_viewpoint([10,10,10])
# vfh_container.testvfh2([10,10,10])
vfh_container.compute_vpc([10,10,10])

# st = time.perf_counter()
# for i in range(1000): 
#   vfh_container.testvfh2([10,10,10])
# print(time.perf_counter() - st); 






(1390, 3) (1390, 3)
After initializaiton
[[26.198 36.691 27.022]
 [26.539 36.543 27.084]
 [27.075 35.093 28.329]
 ...
 [25.278 32.841 25.679]
 [25.278 33.361 25.679]
 [25.159 33.101 26.13 ]]
[-20.25565252 -25.68393525 -18.78254748]
[-0.53701134 -0.68092422 -0.49795685] (3,)
[0.52718426 0.5938493  0.50031737 ... 0.83792112 0.52964206 0.53701134]
0.5271842627685688
0.5938492969508882
0.5003173683467622
0.8732540340468018
0.7816073270795973
0.8173920929446075
0.5668221641191383
0.5322932213596699
0.8792227776861242
-0.021205475849279898
-0.4741820873481106
0.1945349586270783
-0.20987448735078806
-0.7908499102819864
-0.5163115634085433
-0.34434778788964315
-0.7104123318205928
0.5334592254645619
-0.1406797840771912
0.33179730528938844
0.7582596151272685
-0.41936701288798606
-0.00335659706607333
0.24561628415221654
-0.06160551709695791
0.23233380824172234
0.4985848355839647
0.28718537433198266
0.46085053480574456
0.7015156956252413
0.536571099753376
0.43618392046274773
0.5128476647820629
0.4

array([0.        , 0.        , 0.00071942, 0.00143885, 0.00215827,
       0.00071942, 0.00143885, 0.00215827, 0.00503597, 0.00143885,
       0.00143885, 0.        , 0.00431655, 0.0057554 , 0.00431655,
       0.00791367, 0.0028777 , 0.00431655, 0.00359712, 0.0057554 ,
       0.00647482, 0.00647482, 0.00647482, 0.00719424, 0.00359712,
       0.01007194, 0.01007194, 0.01151079, 0.00863309, 0.00647482,
       0.01079137, 0.00719424, 0.01079137, 0.00791367, 0.00863309,
       0.00719424, 0.01079137, 0.01151079, 0.00863309, 0.01223022,
       0.01007194, 0.01366906, 0.00863309, 0.01366906, 0.01366906,
       0.00719424, 0.01079137, 0.00863309, 0.00647482, 0.01151079,
       0.01007194, 0.00647482, 0.01438849, 0.01079137, 0.00863309,
       0.00935252, 0.01079137, 0.01510791, 0.00863309, 0.01726619,
       0.01366906, 0.01438849, 0.01582734, 0.01223022, 0.01079137,
       0.00791367, 0.00935252, 0.01007194, 0.01582734, 0.00791367,
       0.01582734, 0.01151079, 0.01726619, 0.0057554 , 0.01223

In [3]:

import open3d as o3d
import matplotlib.pyplot as plt 
import open3d as o3d
import numpy as np
from scipy import spatial

In [36]:
np.arange(1,4)*np.arange(1,4)

np.arange(1,4).dot(np.arange(1,4))


14

In [1]:
import numpy as np
from BetaPose import utils

def uv(vector):
    return vector / np.linalg.norm(vector)

def ea_vectors(v1, v2):
    # Normalize both vectors
    v1_unit = v1/np.linalg.norm(v1)
    v2_unit = v2/np.linalg.norm(v2)

    # Calculate the cross product and dot product of the vectors
    cross = np.cross(v1_unit, v2_unit)
    dot = np.dot(v1_unit, v2_unit)

    # Calculate the angle between the vectors
    angle = np.arccos(np.clip(dot, -1.0, 1.0))

    # Calculate the axis of rotation
    axis = cross / np.linalg.norm(cross)

    # Calculate the quaternion
    qw = np.cos(angle / 2)
    qx, qy, qz = axis * np.sin(angle / 2)

    # Convert quaternion to Euler angles
    roll = np.arctan2(2 * (qw * qx + qy * qz), 1 - 2 * (qx * qx + qy * qy))
    pitch = np.arcsin(2 * (qw * qy - qz * qx))
    yaw = np.arctan2(2 * (qw * qz + qx * qy), 1 - 2 * (qy * qy + qz * qz))

    return roll, pitch, yaw

v1 = np.array([1, 0, 0])
v2 = np.array([3, 2, 1])

roll, pitch, yaw = ea_vectors(v1, v2)
print(f"Roll: {roll}, Pitch: {pitch}, Yaw: {yaw}")

Roll: -0.0823726211170212, Pitch: -0.27054976297857286, Yaw: 0.5880026035475675


In [70]:
tm = utils.calc_tm(roll, pitch, yaw); 
print(tm)
totrans = np.array([v2])
print(totrans)

ret = utils.transform_pcd(totrans, tm)
print(np.round(uv(ret),2), "diff", np.round((uv(ret) / uv(v1)), 2))

[[ 0.80178373 -0.53452248 -0.26726124  0.        ]
 [ 0.57111626  0.81703113  0.07928651  0.        ]
 [ 0.17598033 -0.21620787  0.96035675  0.        ]
 [ 0.          0.          0.          1.        ]]
[[3 2 1]]
[[0.29 0.92 0.28]] diff [[0.29  inf  inf]]


/tmp/ipykernel_10361/3694480922.py:7: RuntimeWarning: divide by zero encountered in true_divide
  print(np.round(uv(ret),2), "diff", np.round((uv(ret) / uv(v1)), 2))


In [39]:
import numpy as np

def compute_angles(v1, v2):
    # Normalize vectors
    v1_norm = v1 / np.linalg.norm(v1)
    v2_norm = v2 / np.linalg.norm(v2)

    # Compute the cross product of the two vectors
    cross = np.cross(v1_norm, v2_norm)

    # Compute the dot product of the two vectors
    dot = np.dot(v1_norm, v2_norm)

    # Compute the magnitude of the cross product and the angle between the two vectors
    mag_cross = np.linalg.norm(cross)
    angle = np.arccos(dot)

    # Compute the rotation matrix around the cross product vector
    if mag_cross != 0:
        cross_norm = cross / mag_cross
        cos_angle = np.cos(angle)
        sin_angle = np.sin(angle)
        R = np.array([[cos_angle + cross_norm[0]**2*(1-cos_angle), cross_norm[0]*cross_norm[1]*(1-cos_angle) - cross_norm[2]*sin_angle, cross_norm[0]*cross_norm[2]*(1-cos_angle) + cross_norm[1]*sin_angle],
                      [cross_norm[1]*cross_norm[0]*(1-cos_angle) + cross_norm[2]*sin_angle, cos_angle + cross_norm[1]**2*(1-cos_angle), cross_norm[1]*cross_norm[2]*(1-cos_angle) - cross_norm[0]*sin_angle],
                      [cross_norm[2]*cross_norm[0]*(1-cos_angle) - cross_norm[1]*sin_angle, cross_norm[2]*cross_norm[1]*(1-cos_angle) + cross_norm[0]*sin_angle, cos_angle + cross_norm[2]**2*(1-cos_angle)]])
    else:
        R = np.eye(3)

    # Compute the roll, pitch, and yaw angles from the rotation matrix
    yaw = np.arctan2(R[1, 0], R[0, 0])
    pitch = np.arctan2(-R[2, 0], np.sqrt(R[2, 1]**2 + R[2, 2]**2))
    roll = np.arctan2(R[2, 1], R[2, 2])

    return roll, pitch, yaw
  
eas = compute_angles(v1,v2)
tm = utils.calc_tm(*eas); 
ret = utils.transform_pcd(np.array([v2]), tm)
print(ret.round(2))



[[-0.82  0.96  0.64]]


In [3]:
import numpy as np 
np.ravel([[1, 2, 3], [4, 5, 6], [7, 8, 9]]).__len__() %3

0